In [ ]:
import numpy as np
import pandas as pd
import operator

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [ ]:
# Load 99 post office locations into a data frame, specifying data types and disabling zero suppress.
dfpo2 = pd.read_csv('2019-01_post-office-Hokkaido1e.csv',
    dtype={'po_code':str,'po_name':str,'longitude':np.float64,'latitude':np.float64})
dfpo2.head()

In [ ]:
# Create a data frame excluding major cities and popular places I already visited.
dfpo4 = dfpo2[
    dfpo2['po_name'].apply(str.__contains__, str, ['Sapporo']).apply(operator.__not__)
]
dfpo5 = dfpo4[
    dfpo4['po_name'].apply(str.__contains__, str, ['Otaru']).apply(operator.__not__)
]
dfpo6 = dfpo5[
    dfpo5['po_name'].apply(str.__contains__, str, ['Yoichi']).apply(operator.__not__)
]
dfpo7 = dfpo6[
    dfpo6['po_name'].apply(str.__contains__, str, ['Muroran']).apply(operator.__not__)
]
dfpo8 = dfpo7[
    dfpo7['po_name'].apply(str.__contains__, str, ['Tomakomai']).apply(operator.__not__)
]
dfpo8.head()

In [ ]:
# Print 4 east-end locations.
dfpo8.sort_values(['longitude'],ascending=False).head(4)

In [ ]:
# Print 2 west-end locations.
dfpo8.sort_values(['longitude']).head(2)

In [ ]:
# Print 2 north-end locations.
dfpo8.sort_values(['latitude'],ascending=False).head(2)

In [ ]:
# Calculate the epicenter of new 78 locations and display a map
longitude,latitude = dfpo8[['longitude','latitude']].mean().tolist()
map3 = folium.Map(location=[latitude, longitude], zoom_start=7)

# Display a map with circle markers (po_code)
for lat, lon, po_name in zip(dfpo8['latitude'],dfpo8['longitude'],dfpo8['po_name']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=po_name,
        color='yellow',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map3)
map3

In [ ]:
# Calculate distance from latitude and longitude
from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
def dist_lat_long(latd1, lond1, latd2, lond2):
    R = 6373.0
    # convert to radian
    latr1 = radians(latd1)
    lonr1 = radians(lond1)
    latr2 = radians(latd2)
    lonr2 = radians(lond2)
    dlonr = lonr2 - lonr1
    dlatr = latr2 - latr1

    a = sin(dlatr/2)**2 + cos(latr1) * cos(latr2) * sin(dlonr/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R*c

In [ ]:
# unit for latitude/longitude: degrees
dest1 = 'Hidaka Meguro'
origin1 = 'Erimo Misaki'
lond1, latd1 = dfpo8[dfpo8['po_name']==dest1][['longitude','latitude']].get_values()[0].tolist()
# print (lond1, latd1) 
lond2, latd2 = dfpo8[dfpo8['po_name']==origin1][['longitude','latitude']].get_values()[0].tolist()
print("Distance from {} to {}: {:.1f} km".format(
    origin1, dest1, dist_lat_long(latd1, lond1, latd2, lond2)))

In [ ]:
# Distanace from Major cities (those not in our data frame dfpo8)
dest = 'Nakoma'
lond2, latd2 = dfpo8[dfpo8['po_name']==dest][['longitude','latitude']].get_values()[0].tolist()
# print (lond2, latd2)

major_cities = ['Sapporo', 'Hakodate', 'Otaru', 'Muroran', 'Tomakomai', 'Asahikawa']

for city in major_cities:
    address = city + ", Japan"
    # print (address)
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latd1 = location.latitude
    lond1 = location.longitude
    # print (lond1, latd1)
    print("Distance from {} to {}: {:.1f} km".format(
        city, dest, dist_lat_long(latd1, lond1, latd2, lond2)))
    